In [19]:
import pandas as pd
marathon_2017 = pd.read_csv("marathon_results_2017.csv")
marathon_2017.head(2)

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,1:16:59,1:33:01,1:48:19,2:02:53,0:04:57,-,2:09:37,1,1,1
1,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,1:16:59,1:33:01,1:48:19,2:03:14,0:04:58,-,2:09:58,2,2,2


In [20]:
marathon_2017 = marathon_2017.drop(['Unnamed: 0', 'Bib', 'Name', 'Age', 'M/F', 'City', 'State', 'Country', \
       'Citizen', 'Unnamed: 9', 'Half', 'Pace', 'Proj Time', 'Official Time', 'Overall', 'Gender', 'Division' \
                                   ], axis='columns')
marathon_2017.head(2)

,5K,10K,15K,20K,25K,30K,35K,40K
0,0:15:25,0:30:28,0:45:44,1:01:15,1:16:59,1:33:01,1:48:19,2:02:53
1,0:15:24,0:30:27,0:45:44,1:01:15,1:16:59,1:33:01,1:48:19,2:03:14


In [21]:
import numpy as np
marathon_2017['5K'] = pd.to_timedelta(marathon_2017['5K'])
marathon_2017['10K'] = pd.to_timedelta(marathon_2017['10K'])
marathon_2017['15K'] = pd.to_timedelta(marathon_2017['15K'])
marathon_2017['20K'] = pd.to_timedelta(marathon_2017['20K'])
marathon_2017['25K'] = pd.to_timedelta(marathon_2017['25K'])
marathon_2017['30K'] = pd.to_timedelta(marathon_2017['30K'])
marathon_2017['35K'] = pd.to_timedelta(marathon_2017['35K'])
marathon_2017['40K'] = pd.to_timedelta(marathon_2017['40K'])

marathon_2017['5K'] = marathon_2017['5K'].astype('m8[s]').astype(np.int64)
marathon_2017['10K'] = marathon_2017['10K'].astype('m8[s]').astype(np.int64)
marathon_2017['15K'] = marathon_2017['15K'].astype('m8[s]').astype(np.int64)
marathon_2017['20K'] = marathon_2017['20K'].astype('m8[s]').astype(np.int64)
marathon_2017['25K'] = marathon_2017['25K'].astype('m8[s]').astype(np.int64)
marathon_2017['30K'] = marathon_2017['30K'].astype('m8[s]').astype(np.int64)
marathon_2017['35K'] = marathon_2017['35K'].astype('m8[s]').astype(np.int64)
marathon_2017['40K'] = marathon_2017['40K'].astype('m8[s]').astype(np.int64)
marathon_2017.head(3)

check_time = 7200 # 기준시간 설정
Lat =  0
Long = 0  # 두 시간 경과타임에 위치한 곳의 위도와 경도를 저장할 변수
# 각 지점(5K, 10K, 15K, ...) 위도와 경도
points = [[42.247835,-71.474357],[42.274032,-71.423979],[42.282364,-71.364801],\
          [42.297870,-71.284260],[42.324830,-71.259660],[42.345680,-71.215169],\
          [42.352089,-71.124947],[42.351510,-71.086980]]

marathon_2017_location = pd.DataFrame(columns=['Lat','Long'])

for index, record in marathon_2017.iterrows():
    if record['40K']<check_time: 
        Lat = points[7][0]
        Long = points[7][1]
    elif record['35K']<check_time: 
        Lat = points[6][0]
        Long = points[6][1]
    elif record['30K']<check_time:  
        Lat = points[5][0]
        Long = points[5][1]
    elif record['25K']<check_time:  
        Lat = points[4][0]
        Long = points[4][1]
    elif record['20K']<check_time:  
        Lat = points[3][0]
        Long = points[3][1]
    elif record['15K']<check_time:  
        Lat = points[2][0]
        Long = points[2][1]
    elif record['10K']<check_time:  
        Lat = points[1][0]
        Long = points[1][1]
    else:
        Lat = points[0][0]
        Long = points[0][1]
    marathon_2017_location = marathon_2017_location.append({'Lat':Lat, 'Long':Long}, ignore_index=True)
    
marathon_count = marathon_2017_location.groupby(['Lat','Long']).size().reset_index(name='Count')
marathon_count = marathon_2017_location.groupby(['Lat','Long'])['Long'].count().reset_index(name='Count')
marathon_count

,Lat,Long,Count
0,42.274032,-71.423979,49
1,42.282364,-71.364801,4435
2,42.297870,-71.284260,13866
3,42.324830,-71.259660,7261
4,42.345680,-71.215169,737
5,42.351510,-71.086980,6
6,42.352089,-71.124947,56


In [22]:
import folium
from folium.plugins import HeatMap

In [23]:
marathon_map = folium.Map(location = [42.324830, -71.25960], zoom_start=11)
# location  : 표시될 지도의 중점 좌표
HeatMap(marathon_count, radius=20).add_to(marathon_map)
# HeatMap : 값에 따라 표시되는 색깔이 달라지는 표식
# 값이 클수록 붉은색, 작을수록 파란색 : 열의 색상표시
marathon_map